<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/CV2_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 07/30/2021-1
from __future__ import absolute_import
from IPython.display import clear_output

import glob, os, shutil
from os.path import *

try: 
    from google.colab import drive, files
    from google.colab.patches import cv2_imshow
except Exception as err:
    pass

try: 
    from matplotlib import pyplot as plt
except Exception as err:
    %pip install matplotlib
    from matplotlib import pyplot as plt

try: 
    import cv2
    from cv2 import COLOR_BGR2RGBA
except Exception as err:
    print(err)
    %pip install opencv-python-headless # works.
    # %pip install opencv-contrib-python-headless
    import cv2
    from cv2 import COLOR_BGR2RGBA, IMREAD_UNCHANGED

import numpy as np

if exists('/content/sample_data'):
    shutil.rmtree('/content/sample_data')

contentPth = os.getcwd()
tensorflowImagesPath = join(contentPth, 'TensorflowImages')
resizedImagesPath = join(contentPth, 'TensorflowResizedImages')

if exists(resizedImagesPath):
    pass
    shutil.rmtree(resizedImagesPath)

try:
    drive.flush_and_unmount()
    drive.mount('/content/gdrive/', force_remount=True)
    mydrivePth = '/content/gdrive/MyDrive'
    os.chdir(mydrivePth)
    gdriveTFImagesPth = join(mydrivePth, 'TensorflowImages')
    gdriveResizedImagesPth = join(mydrivePth, 'TensorflowResizedImages')
    
    os.chdir(mydrivePth)
    from TarfileFunctions import *
    from BashColors import C
    
    if not exists(tensorflowImagesPath):
        shutil.copytree(gdriveTFImagesPth, tensorflowImagesPath)
        # tff.listTarfiles()
        os.chdir(contentPth)
        # tff.extractTarfiles('Juno_Original_Images.tar.gz')

    gdrive_resized_images = '/content/gdrive/MyDrive/TensorflowResizedImages'
    if not exists(resizedImagesPath):
        try: shutil.copytree(gdrive_resized_images, resizedImagesPath)
        except Exception as err:
            print(err)

    os.chdir(contentPth)

except Exception as err:
    # print(err)
    os.chdir(contentPth)
    from TarfileFunctions import *
    from BashColors import C
    tarfilePath=join(contentPth, 'Juno_Original_Images.tar.gz')
    if exists(tarfilePath) and not exists(tensorflowImagesPath):
        tff.extractTarfiles('Juno_Original_Images.tar.gz')

rootPth = expanduser("~")

os.chdir(contentPth)
print(f'cv2: {C.IBlue}{cv2.__version__}{C.ColorOff}')
print(f'cwd: {C.IBlue}{os.getcwd()}')

In [ ]:
os.chdir(tensorflowImagesPath)
imageGlob = glob.glob('**', recursive=True)
pathList=[]
count=0
for path in sorted(imageGlob):
    fullPath = abspath(path)
    if isfile(path) and not path.__contains__('backgroundImage224.png'):
        pathList.append(fullPath)
        print(count, fullPath)
        count+=1
clear_output()

In [ ]:
def matPlotCompareTwoImages(thisImage, compareImage):
    thisImage = cv2.cvtColor(thisImage, cv2.COLOR_BGR2BGRA)
    compareImage = cv2.cvtColor(compareImage, cv2.COLOR_BGR2BGRA)
    # Plot the images
    fig=plt.figure()
    ax1 = fig.add_subplot(1,2,1)
    ax1.imshow(thisImage)
    ax1.axis('off')
    ax1.set_title('Original Image')

    ax2 = fig.add_subplot(1,2,2)
    ax2.imshow(compareImage)
    ax2.axis('off')
    ax2.set_title('New Image')
    plt.show()

def cv2ResizeImage(thisImage, newSize=(224,224)):
    resized_image = cv2.resize(thisImage,newSize,interpolation=cv2.INTER_CUBIC)
    return resized_image

def cv2CreateImageWithColor(pxColor, silent=True):
    bgImagePath = join(contentPth, 'TensorflowImages/backgroundImage224.png')
    bgImage = cv2.imread(bgImagePath, cv2.IMREAD_UNCHANGED)
    save_path = join(contentPth, 'bgImage.png')
    # print(save_path)
    bgImage = np.copy(bgImage)

    for px in bgImage:
        bgImage[:] = pxColor
        cv2.waitKey(100)
        
    cv2.imwrite(save_path, bgImage)
    if not silent:
        print(f'shape: {bgImage.shape}')
        # cv2_imshow(bgImage)
        matPlotCompareTwoImages(bgImage, bgImage)
    return save_path

def cv2ZoomOutImage(imagePath):
    img = cv2.imread(imagePath, cv2.IMREAD_UNCHANGED)


def cv2Rotation(thisImage, angle=0, rotPoint=None, newScale=1, silent=True):
    ''''''
    thisImage = np.copy(thisImage)
    zeroPixel = thisImage[[0][0]]
    width, height = thisImage.shape[:2]
    if rotPoint == None:
        rotPoint = width//2, height//2
        rotMat = cv2.getRotationMatrix2D(rotPoint, angle, scale=newScale)
        dimentions = width, height
        rotImage = cv2.warpAffine(thisImage, rotMat, dimentions)
        rotImageZeroPixel = rotImage[0][0]
        if not silent:
            print('cv2Rotation()')
            matPlotCompareTwoImages(thisImage, rotImage)
            cv2.waitKey(100)
        cv2.waitKey(100)
        cv2.destroyAllWindows()
        return rotImage

    def cv2Translate(thisImage, x=0, y=0, silent=True):
        ''''''
        thisImage = np.copy(thisImage)
        translateMatrix = np.float32([[1,0,x],[0,1,y]])
        dimentions = (thisImage.shape[1], thisImage.shape[0])
        newImage = cv2.warpAffine(thisImage, translateMatrix, dimentions)
        if not silent:
            print('cv2Translate()')
            matPlotCompareTwoImages(thisImage, newImage)
            cv2.waitKey(100)
        cv2.waitKey(100)
        cv2.destroyAllWindows()
        return newImage


originalImage = cv2.imread(pathList[0], cv2.IMREAD_UNCHANGED)
originalImage = cv2ResizeImage(originalImage)
zoomImage = cv2ZoomOutImage(pathList[0])

# _ = cv2Translate(zoomImage)

cv2.waitKey(100)
cv2.destroyAllWindows()

In [ ]:
help(cv2Rotation)

In [ ]:
_ = cv

In [ ]:
count=0
for pth in sorted(pathList):
    imagePath = pth
    enemyImage = cv2.imread(imagePath, cv2.IMREAD_UNCHANGED)
    enemyImage = cv2.resize(enemyImage, (224, 224), 
                            interpolation=cv2.INTER_CUBIC)
    print(f'pathList[{count}]')
    count+=1
    print(imagePath)

    zeroPixel = enemyImage[0][0]

    bgImagePath = cv2CreateImageWithColor(pxColor=zeroPixel)
    bgImage = cv2.imread(bgImagePath, cv2.IMREAD_UNCHANGED)
    cv2_imshow(enemyImage)

    imageSecondary = cv2.hconcat([bgImage,bgImage,bgImage])
    imagePrimary = cv2.hconcat([bgImage,enemyImage,bgImage])
    zoomImage = cv2.vconcat([imageSecondary,imagePrimary,imageSecondary])
    zoomImage = cv2ResizeImage(thisImage=zoomImage)

    print(zoomImage.shape)
    cv2_imshow(zoomImage)
    matPlotCompareTwoImages(enemyImage, zoomImage)
    cv2.waitKey(500)
    cv2.destroyAllWindows()
    print()

In [ ]:
_=cv2Rotation(thisImage=originalImage, silent=False)